In [1]:
!pip install pygame

In [5]:
import pygame
import os
import json

# Pygame hasieratu
pygame.init()

# Irudiak karpeta batetik irakurri eta gordetzeko funtzioa
def load_images(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            images.append((filename, pygame.image.load(os.path.join(folder_path, filename))))
    return images

# Irudien karpeta adierazi eta kargatu
image_folder = "images"
image_list = load_images(image_folder)
total_images = len(image_list)
current_image_index = 0

# Pantailaren tamaina adierazi (irudien resolution ikusi eta aldatu)
window_width = 1765
window_height = 993
screen = pygame.display.set_mode((window_width, window_height))

# Irudien tamaina eskalatzeko funtzioa
def calculate_scale_factor(image):
    image_width, image_height = image.get_size()
    return min(window_width / image_width, window_height / image_height)

# botoien koordenatuak gordeko diren JSON fitxategia sortu
json_filename = "button_coordinates.json"
button_coordinates_data = {}

# irudi batean botoi bat badago, marraztu botoia
def draw_buttons(image_index):
    screen.fill((255, 255, 255))

    # marraztu irudia
    scaled_image = pygame.transform.scale(image_list[image_index][1],
                                          (int(image_list[image_index][1].get_width() * calculate_scale_factor(image_list[image_index][1])),
                                           int(image_list[image_index][1].get_height() * calculate_scale_factor(image_list[image_index][1]))))
    screen.blit(scaled_image, (0, 0))

    # marraztu botoiak
    if image_list[image_index][0] in button_coordinates_data:
        for button in button_coordinates_data[image_list[image_index][0]]:
            top_left = (button["top_left"]["x"], button["top_left"]["y"])
            bottom_right = (button["bottom_right"]["x"], button["bottom_right"]["y"])
            pygame.draw.rect(screen, (0, 255, 0), (top_left, (bottom_right[0] - top_left[0], bottom_right[1] - top_left[1])), 2)

    pygame.display.flip()

# botoiak nora joan diren marrazteko
mouse_pressed = False
top_left_coordinates = None

# irudiak pantailaratu hasi aurretik
draw_buttons(current_image_index)

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            # sagua sakatu denean, botoiaren goiko ezkerreko puntua gorde
            if not mouse_pressed:
                x, y = event.pos
                top_left_coordinates = (x, y)
                mouse_pressed = True
        elif event.type == pygame.MOUSEBUTTONUP:
            # sagua askatu denean, botoiaren beheko eskuineko puntua gorde
            if mouse_pressed:
                x, y = event.pos
                bottom_right_coordinates = (x, y)
                mouse_pressed = False

                # botoiaren koordenatuak gorde
                if image_list[current_image_index][0] not in button_coordinates_data:
                    button_coordinates_data[image_list[current_image_index][0]] = []

                button_coordinates_data[image_list[current_image_index][0]].append({
                    "top_left": {"x": top_left_coordinates[0], "y": top_left_coordinates[1]},
                    "bottom_right": {"x": bottom_right_coordinates[0], "y": bottom_right_coordinates[1]}
                })

                # botoiak marraztu
                draw_buttons(current_image_index)

        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:  # Aurreko irudia
                current_image_index = (current_image_index - 1) % total_images
                draw_buttons(current_image_index)
            elif event.key == pygame.K_RIGHT:  # Hurrengo irudia
                current_image_index = (current_image_index + 1) % total_images
                draw_buttons(current_image_index)

pygame.quit()
